In [1]:
import os
from datetime import datetime, time, date
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [33]:
data_path = r'D:\SBP and Seismic\abp49_SBP\abp49_medu_taran_11'

utm_coords = True
year = 2022

# track
save_to = r'D:\abp49_track111.txt'

# replica
replica_path = r'D:\SBP and Seismic\replica_3.txt'

pos_files = fes.file_ext_search('.txt', data_path)
print(len(pos_files))

Searching *.txt files in directory:D:\SBP and Seismic\abp49_SBP\abp49_medu_taran_11
9


In [34]:
# Reading raw pos
@dataclass
class SegyPosFile:
    name: str
    fin_traceno: int = 0
    datetime: list = field(default_factory=list)
    traceno: list = field(default_factory=list)
    cdp_x: list = field(default_factory=list)
    cdp_x_smoothed: list = field(default_factory=list)
    cdp_y_smoothed: list = field(default_factory=list)
    cdp_x_cartesian_smoothed: list = field(default_factory=list)
    cdp_y_cartesian_smoothed: list = field(default_factory=list)
    cdp_y: list = field(default_factory=list)
    year: list = field(default_factory=list)
    day: list = field(default_factory=list)
    hour: list = field(default_factory=list)
    minute: list = field(default_factory=list)
    second: list = field(default_factory=list)

segy_pos_objs = []
bad_data_dict = {}
fine_data_dict = {}


for pos_file in pos_files:
    segy_name = os.path.splitext(os.path.basename(pos_file))[0][:-7]
    print(segy_name)
    pos_obj = SegyPosFile(name=segy_name)
    number = 0
    
    has_error = False
    was_before = False
    with open(pos_file, 'r') as file1:
        file_content = file1.read().splitlines()
        
        for line in file_content[1:]:
            line_content = line.split()

            try:
                
                if utm_coords:
                    if int(line_content[3]) != year:
                        raise RuntimeError('BadYear')
                    
                    elif int(int(line_content[4])) > 370 and int(line_content[4]) < 0:
                        raise RuntimeError('BadDay')
                    
                    elif float(line_content[1]) < 50000 and float(line_content[1]) > 500000:
                        raise RuntimeError('BadCDP_X')
                    
                    elif float(line_content[2]) < 200000 and float(line_content[2]) > 8000000:
                        raise RuntimeError('BadCDP_Y')

                else:
                    if int(line_content[3]) != year:
                        raise RuntimeError('BadYear')
                    
                    elif int(int(line_content[4])) > 370 and int(line_content[4]) < 0:
                        raise RuntimeError('BadDay')
                    
                    elif float(line_content[1]) < 15.0 and float(line_content[1]) > 50.0:
                        raise RuntimeError('BadCDP_X')
                    
                    elif float(line_content[2]) < 30.0 and float(line_content[2]) > 70.0:
                        raise RuntimeError('BadCDP_Y')
                
                ar = time(hour=int(line_content[5]),
                        minute=int(line_content[6]), second=int(line_content[7]))
            except:
                number += 1
                has_error = True
                if not was_before:
                    bad_data_dict['segy_name'] = pos_obj.traceno[-1]
                    was_before = True
                    
            else:
                pos_obj.datetime.append(f'{line_content[3]}-{line_content[4]}T{line_content[5]}:{line_content[6]}:{line_content[7]}')
                pos_obj.traceno.append(int(line_content[0]))
                pos_obj.cdp_x.append(float(line_content[1]))
                pos_obj.cdp_y.append(float(line_content[2]))
                pos_obj.year.append(int(line_content[3]))
                pos_obj.day.append(int(line_content[4]))
                pos_obj.hour.append(int(line_content[5]))
                pos_obj.minute.append(int(line_content[6]))
                pos_obj.second.append(int(line_content[7]))
                
        fine_data_dict[segy_name] = pos_obj.traceno[-1]
        
        if has_error:
            print(f'Number of bad lines in {segy_name}: {number}')
            
        segy_pos_objs.append(pos_obj)

# Sorting files
SLF_objs = []
PHF_objs= []

for segy_pos_obj in segy_pos_objs:
    if 'SLF' in segy_pos_obj.name:
        SLF_objs.append(segy_pos_obj)
    elif 'PHF' in segy_pos_obj.name:
        PHF_objs.append(segy_pos_obj)

# Track Processing
from pyproj import Proj, CRS, Transformer

crs_wgs84 = CRS.from_epsg(4326)
# crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)
transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

with open(save_to, 'w') as file2:
    file2.write('num_o,num_i,name,datetime,traceno,cdp_x,cdp_y,year,day,hour,minute,second\n')
    num_o = 0
    num_f = 0
    for segy_pos_obj in SLF_objs:
        window_length = 201
        file_length = len(segy_pos_obj.cdp_x)
        
        loop = True
        while loop:
            if window_length > file_length/4:
                window_length = int(window_length/4)
            elif window_length < 10:
                loop = False
            else:
                loop = False
        
        if window_length < 10:
            print(f'Can not smooth file {segy_pos_obj.name}')
            
            if utm_coords:
                cartesian_x = segy_pos_obj.cdp_x
                cartesian_y = segy_pos_obj.cdp_y
            else:
                cartesian_x, cartesian_y = transformer.transform(segy_pos_obj.cdp_x, segy_pos_obj.cdp_y)
                
            if '_not_smoothed' in segy_pos_obj.name:
                pass
            else:
                segy_pos_obj.name = segy_pos_obj.name + '_not_smoothed'
            
            segy_pos_obj.cdp_x_cartesian_smoothed = cartesian_x
            segy_pos_obj.cdp_y_cartesian_smoothed = cartesian_y
        
        else:
            segy_pos_obj.cpd_x_smoothed = signal.savgol_filter(segy_pos_obj.cdp_x,window_length,3)
            segy_pos_obj.cpd_y_smoothed = signal.savgol_filter(segy_pos_obj.cdp_y,window_length,3)
            
            if utm_coords:
                cartesian_x = segy_pos_obj.cpd_x_smoothed
                cartesian_y = segy_pos_obj.cpd_y_smoothed
            else:
                cartesian_x, cartesian_y = transformer.transform(segy_pos_obj.cpd_x_smoothed, segy_pos_obj.cpd_y_smoothed)
        
            segy_pos_obj.cdp_x_cartesian_smoothed = cartesian_x.tolist()
            segy_pos_obj.cdp_y_cartesian_smoothed = cartesian_y.tolist()
        
        for num_i,traceno in enumerate(segy_pos_obj.traceno):
            file2.write(f'{num_o},{num_i},{segy_pos_obj.name},{segy_pos_obj.datetime[num_i]},{segy_pos_obj.traceno[num_i]},{segy_pos_obj.cdp_x_cartesian_smoothed[num_i]},')
            file2.write(f'{segy_pos_obj.cdp_y_cartesian_smoothed[num_i]},{segy_pos_obj.year[num_i]},{segy_pos_obj.day[num_i]},{segy_pos_obj.hour[num_i]},{segy_pos_obj.minute[num_i]},')
            file2.write(f'{segy_pos_obj.second[num_i]}\n')
            num_o += 1
        num_f += 1
        print(f'File {segy_pos_obj.name} is done {num_f} of {len(segy_pos_objs)}')


test-1_W2-SLF_ABP49_SLF2206061243_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061252_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061300_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061308_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061309_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061316_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061323_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061331_UTM-34_car
test-1_W2-SLF_ABP49_SLF2206061338_UTM-34_car
File test-1_W2-SLF_ABP49_SLF2206061243_UTM-34_car is done 1 of 9
File test-1_W2-SLF_ABP49_SLF2206061252_UTM-34_car is done 2 of 9
File test-1_W2-SLF_ABP49_SLF2206061300_UTM-34_car is done 3 of 9
File test-1_W2-SLF_ABP49_SLF2206061308_UTM-34_car is done 4 of 9
File test-1_W2-SLF_ABP49_SLF2206061309_UTM-34_car is done 5 of 9
File test-1_W2-SLF_ABP49_SLF2206061316_UTM-34_car is done 6 of 9
File test-1_W2-SLF_ABP49_SLF2206061323_UTM-34_car is done 7 of 9
File test-1_W2-SLF_ABP49_SLF2206061331_UTM-34_car is done 8 of 9
File test-1_W2-SLF_ABP49_SLF2206061338_UTM-34_car is done 9 of 9


In [35]:
# Replica
sgy_files = fes.file_ext_search('.sgy', data_path)

for sgy_file in sgy_files:
    
    filename = os.path.splitext(os.path.basename(sgy_file))[0]
    
    for slf_obj in SLF_objs:
        if slf_obj.name == filename:
            with open(os.path.join(data_path, filename + '_proc.txt'), 'w') as file3:
                file3.write(f'TraceNo\tCDPX\tCDPY\tBADTRACESTART\n')
                for num, cdp_x in enumerate(slf_obj.cdp_x_cartesian_smoothed):
                    file3.write(f'{slf_obj.traceno[num]}\t{slf_obj.cdp_x_cartesian_smoothed[num]}\t{slf_obj.cdp_y_cartesian_smoothed[num]}\n')

has_header = False

if os.path.exists(replica_path):
    has_header = True
    print('file exists')

with open(replica_path, 'a') as file4:
    if not has_header:
        file4.write('prof_folder\tfile_name\tfile_name_proc\tprof_name\n')
    
    for sgy_file in sgy_files:
        filename = os.path.splitext(os.path.basename(sgy_file))[0]
        prof_folder = data_path
        prof_name = os.path.split(data_path)[1]
        filename_proc = filename + '_proc'
        
        file4.write(f'{prof_folder}\t{filename}\t{filename_proc}\t{prof_name}\n')


Searching *.sgy files in directory:D:\SBP and Seismic\abp49_SBP\abp49_medu_taran_11
file exists
